In [35]:
import os
import glob
from docx import Document
import pytesseract
from pdf2image import convert_from_path
import os

def convert_docx_to_txt(docx_path):
    """
    Читает документ .docx и возвращает его содержимое в виде строки.
    """
    doc = Document(docx_path)
    full_text = [para.text for para in doc.paragraphs]
    text = "\n".join(full_text)
    return text

def ocr_pdf(pdf_path):
    """
    Takes a .pdf document and returns extracted text
    """
    os.environ['TESSDATA_PREFIX'] = '/opt/homebrew/share/tessdata'
    pages = convert_from_path(pdf_path, dpi=300)
    extracted_text = ''
    for i, page in enumerate(pages):
        text = pytesseract.image_to_string(page, lang='rus+eng')
        extracted_text += text
    return extracted_text

def get_text_from_file(file_path):
    """
    Извлекает текст из файла с поддержкой форматов .txt, .docx и .pdf.
    """
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()
    
    if ext == ".txt":
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    elif ext == ".docx":
        return convert_docx_to_txt(file_path)
    elif ext == ".pdf":
        return ocr_pdf(file_path)
    else:
        print(f"Неподдерживаемый формат файла: {file_path}")
        return ""

def process_documents(input_dir, output_dir):
    """
    Обрабатывает все файлы из input_dir с расширениями .docx, .pdf и .txt,
    извлекает из них текст и сохраняет в output_dir в виде файлов с нумерацией (1.txt, 2.txt, ...).
    """
    os.makedirs(output_dir, exist_ok=True)
    
    file_patterns = ["*.docx", "*.pdf", "*.txt"]
    files = []
    for pattern in file_patterns:
        files.extend(glob.glob(os.path.join(input_dir, pattern)))
    
    i = 0
    for idx, file_path in enumerate(files, start=1):
        if i > 9:
            break
        print(f"Обработка файла: {file_path}")
        text = get_text_from_file(file_path)
        if text and text.strip():
            output_file = os.path.join(output_dir, f"{idx}.txt")
            with open(output_file, "w", encoding="utf-8") as f:
                f.write(text)
            print(f"Сохранено: {output_file}")
        else:
            print(f"Пустой текст или ошибка при обработке файла: {file_path}")
        i += 1

input_directory = "./data" 
output_directory = "./proc_data"    

process_documents(input_directory, output_directory)


Обработка файла: ./data/Положение о текущем контроле успеваемост..тета «Высшая школа  экономики».docx
Сохранено: ./proc_data/1.txt
Обработка файла: ./data/100723-6 Заместитель проректора Ватолкина Н.Ш. (передоверие Одоевская Е.В.).docx
Сохранено: ./proc_data/2.txt
Обработка файла: ./data/6.18.1-01_010923-26 Итоговый документ.docx
Сохранено: ./proc_data/3.txt
Обработка файла: ./data/6.18.1-01_280223-13 Итоговый документ.docx
Сохранено: ./proc_data/4.txt
Обработка файла: ./data/6.18-01_250624-2 Приложение 1.docx
Сохранено: ./proc_data/5.txt
Обработка файла: ./data/Положение+о+скидках+EPA+2019_ПУ23.05.19.docx
Сохранено: ./proc_data/6.txt
Обработка файла: ./data/Положение о Междурнародном ..ретного права и политики БРИКС.docx
Сохранено: ./proc_data/7.txt
Обработка файла: ./data/7.18.1-01_090822-1 Приложение 3.docx
Сохранено: ./proc_data/8.txt
Обработка файла: ./data/8.3.6.2-08_191022-1 Приложение 1.docx
Сохранено: ./proc_data/9.txt
Обработка файла: ./data/6.18.1-01_300721-5 Приложение 1.do

In [36]:
import os
import glob
import json
import time
import openai
from usecrets import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
prompt = "Ты эксперт по суммаризации текста. Сформируй краткую, четкую и содержательную суммаризацию представленного текста, избегая шаблонных вводных фраз вроде 'Документ содержит' или 'В документе говорится'. Излагай все от третьего лица в нейтрально-официальном стиле. Ты должен написать не более 2-х предложений. Текст: "

def get_summary(text, max_retries=3):
    """
    Отправляет текст в ChatGPT API для получения суммаризации.
    При возникновении ошибок повторяет запрос с экспоненциальной задержкой.
    """
    for attempt in range(max_retries):
        try:
            client = openai.OpenAI()
            response = client.chat.completions.create(
                model="o1-mini",
                messages=[
                    {"role": "user", "content": prompt + text}
                ],
                # temperature=0.3,
                max_completion_tokens=1000
            )
            summary = response.choices[0].message.content.strip()
            return summary
        except Exception as e:
            print(f"Ошибка при обработке текста: {e}. Попытка {attempt + 1} из {max_retries}.")
            time.sleep(2 ** attempt)
    return None

def generate_training_dataset(input_dir, output_file):
    """
    Обходит все .txt файлы в директории input_dir, считывает их содержимое,
    запрашивает суммаризацию через ChatGPT API и сохраняет итоговые пары {"text": ..., "summary": ...}
    в выходной файл output_file в формате JSON Lines.
    """
    txt_files = glob.glob(os.path.join(input_dir, "*.txt"))
    training_data = []
    
    for txt_file in txt_files:
        print(f"Обработка файла: {txt_file}")
        try:
            with open(txt_file, "r", encoding="utf-8") as f:
                text = f.read()
        except Exception as e:
            print(f"Ошибка чтения файла {txt_file}: {e}")
            continue

        if not text.strip():
            print(f"Пустой текст в файле {txt_file}. Пропускаем.")
            continue

        summary = get_summary(text)
        if summary:
            training_data.append({
                "text": text,
                "summary": summary
            })
            print(f"Суммаризация успешно получена для файла: {txt_file}")
        else:
            print(f"Не удалось получить суммаризацию для файла: {txt_file}")
    
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in training_data:
            json_line = json.dumps(entry, ensure_ascii=False)
            f.write(json_line + "\n")
    
    print(f"Обучающий датасет сохранён в файл: {output_file}")


input_directory = "./proc_data"
output_jsonl = "train.jsonl"
generate_training_dataset(input_directory, output_jsonl)


Обработка файла: ./proc_data/10.txt
Суммаризация успешно получена для файла: ./proc_data/10.txt
Обработка файла: ./proc_data/9.txt
Суммаризация успешно получена для файла: ./proc_data/9.txt
Обработка файла: ./proc_data/8.txt
Суммаризация успешно получена для файла: ./proc_data/8.txt
Обработка файла: ./proc_data/5.txt
Суммаризация успешно получена для файла: ./proc_data/5.txt
Обработка файла: ./proc_data/4.txt
Суммаризация успешно получена для файла: ./proc_data/4.txt
Обработка файла: ./proc_data/6.txt
Суммаризация успешно получена для файла: ./proc_data/6.txt
Обработка файла: ./proc_data/7.txt
Суммаризация успешно получена для файла: ./proc_data/7.txt
Обработка файла: ./proc_data/3.txt
Суммаризация успешно получена для файла: ./proc_data/3.txt
Обработка файла: ./proc_data/2.txt
Суммаризация успешно получена для файла: ./proc_data/2.txt
Обработка файла: ./proc_data/1.txt
Суммаризация успешно получена для файла: ./proc_data/1.txt
Обучающий датасет сохранён в файл: training_data.jsonl


In [19]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
client = openai.OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "Ты эксперт по суммаризации текста. Сформируй краткое и информативное резюме для представленного документа."
        },
        {"role": "user", "content": "тестовый текст"}
    ],
    temperature=0.3,
    max_tokens=256
)
summary = response.choices[0].message.content.strip()
summary

'Это тестовый текст, предназначенный для проверки работы системы суммаризации текста.'